In [2]:
import os
import random
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from utils import yaml_config_hook

from modules import SimCLR, LogisticRegression, get_resnet, EarlyStopping
from modules.transformations import TransformsSimCLR

In [3]:
parser = argparse.ArgumentParser(description="SimCLR")
config = yaml_config_hook("./config/config.yaml")
for k, v in config.items():
    parser.add_argument(f"--{k}", default=v, type=type(v))

args_str = '' 
args, _ = parser.parse_known_args(args=args_str)

# args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.device = torch.device('cuda')

print(args.device)

cuda


In [4]:
torch.manual_seed(args.seed)
np.random.seed(args.seed)
#------- added by young ---------
torch.cuda.manual_seed(args.seed)
if args.gpus > 1:
    torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(args.seed)
random.seed(args.seed)

In [5]:
train_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/train', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)
valid_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/train/valid', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)
test_dataset = torchvision.datasets.ImageFolder(
    '/home/opticho/source/SimCLR/datasets/dataset2(3)/test', 
    transform=TransformsSimCLR(size=(args.image_size, args.image_size)).test_transform)

In [6]:
test_dataset # [ [ [image], [label] ] * 835 ]

Dataset ImageFolder
    Number of datapoints: 776
    Root location: /home/opticho/source/SimCLR/datasets/dataset2(3)/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [7]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=args.workers,
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=args.workers,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=args.logistic_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=args.workers,
)

In [8]:
args.model_num = 5
args.resnet = 'resnet50'
args.batch_size = 32

encoder = get_resnet(args.resnet, pretrained=False)
n_features = encoder.fc.in_features

simclr_model = SimCLR(args, encoder, n_features)
model_fp = os.path.join(
    args.model_path, "model{}.tar".format(args.model_num)
)
simclr_model.load_state_dict(torch.load(model_fp, map_location=args.device.type))
simclr_model = simclr_model.to(args.device)
simclr_model.eval()

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [9]:
n_classes = 3
model = LogisticRegression(simclr_model.n_features, n_classes)
model = model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

Linear(in_features=2048, out_features=3, bias=True)


In [9]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(2720, 675, 776)

In [10]:
def inference(loader, simclr_model, device):
    feature_vector = []
    labels_vector = []
    for step, (x, y) in enumerate(loader):
        x = x.to(device)

        # get encoding
        with torch.no_grad():
            h, _, z, _ = simclr_model(x, x)

        h = h.detach()

        feature_vector.extend(h.cpu().detach().numpy())
        labels_vector.extend(y.numpy())

        # if step % 20 == 0:
        #     print(f"Step [{step}/{len(loader)}]\t Computing features...")

    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector


def get_features(simclr_model, train_loader, valid_loader, test_loader, device):
    train_X, train_y = inference(train_loader, simclr_model, device)
    valid_X, valid_y = inference(valid_loader, simclr_model, device)
    test_X, test_y = inference(test_loader, simclr_model, device)
    return train_X, train_y, valid_X, valid_y, test_X, test_y

In [11]:
def create_data_loaders_from_arrays(X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size):
    train = torch.utils.data.TensorDataset(
        torch.from_numpy(X_train), torch.from_numpy(y_train)
    )
    train_loader = torch.utils.data.DataLoader(
        train, batch_size=batch_size, shuffle=False
    )

    valid = torch.utils.data.TensorDataset(
        torch.from_numpy(X_valid), torch.from_numpy(y_valid)
    )
    valid_loader = torch.utils.data.DataLoader(
        valid, batch_size=batch_size, shuffle=False
    )
    
    test = torch.utils.data.TensorDataset(
        torch.from_numpy(X_test), torch.from_numpy(y_test)
    )
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=batch_size, shuffle=False
    )
    return train_loader, valid_loader, test_loader


In [12]:
(train_X, train_y, valid_X, valid_y, test_X, test_y) = get_features(
    simclr_model, train_loader, valid_loader, test_loader, args.device
)

arr_train_loader, arr_valid_loader, arr_test_loader = create_data_loaders_from_arrays(
    train_X, train_y, valid_X, valid_y, test_X, test_y, args.logistic_batch_size
)
patience = 20

Features shape (2720, 2048)
Features shape (675, 2048)
Features shape (776, 2048)


In [15]:
def train_model(args, model, train_loader, valid_loader, criterion, optimizer, patience):
    epochs = args.logistic_epochs
    device = args.device
    valid_loss_min = np.Inf
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
    train_acc, valid_acc = [],[]
    #valid_acc =[]
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    for epoch in range(epochs):

        model.train()
        total_train = 0
        correct_train = 0
        total_valid = 0
        correct_valid = 0
        
        for step, (inputs, labels) in enumerate(train_loader):
            
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            predicted = outputs.argmax(1)
            total_train += labels.nelement()
            correct_train += (predicted == labels).sum().item()
            train_accuracy = correct_train / total_train
        
        model.eval()
            
        with torch.no_grad():
            valid_accuracy = 0
            for inputs, labels in valid_loader:
                
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_losses.append(loss.item())
                # Calculate accuracy
                predicted = outputs.argmax(1)
                total_valid += labels.nelement()
                correct_valid += (predicted == labels).sum().item()
                valid_accuracy = correct_valid / total_valid
            
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        valid_acc.append(valid_accuracy) 
        train_acc.append(train_accuracy)

        # calculate average losses
        
        # print training/validation statistics 
        print(f"Epoch {epoch+1}/{epochs}.. ")
        #print('train Loss: {:.3f}'.format(epoch, loss.item()), "Training Accuracy: %d %%" % (train_accuracy))
        #print('Training Accuracy: {:.6f}'.format(
        #    train_accuracy))
        print('Training Loss: {:.6f} \tValidation Loss: {:.6f} \tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
            train_loss, valid_loss, train_accuracy*100, valid_accuracy*100))
        train_losses = []
        valid_losses = []        
        if valid_accuracy > best_acc:
            best_acc = valid_accuracy
        early_stopping(valid_loss, args, model, optimizer, save=False)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    print('Best val Acc: {:4f}'.format(best_acc*100))  
    # model.load_state_dict(torch.load('checkpoint.pt'))
    # plt.title("Accuracy vs. Number of Training Epochs")
    # plt.xlabel("Training Epochs")
    # plt.ylabel("Accuracy")      
    # plt.plot(train_acc, label='Training acc')
    # plt.plot(valid_acc, label='Validation acc')
    # plt.legend(frameon=False)
    # plt.show()
    return  model, avg_train_losses, avg_valid_losses,  train_acc, valid_acc


In [16]:
args.logistic_epochs = 10000

model, train_loss, valid_loss, train_acc, valid_acc = train_model(
    args, model, 
    arr_train_loader, arr_valid_loader, 
    criterion, optimizer, patience)

if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

out = os.path.join(args.model_path, "downstream_model5.tar")

torch.save(model.state_dict(), out)

Epoch 1/10000.. 
Training Loss: 0.852188 	Validation Loss: 0.756382 	Training Accuracy: 0.000000 	Validation Accuracy: 64.444444
Epoch 2/10000.. 
Training Loss: 0.766821 	Validation Loss: 0.749102 	Training Accuracy: 0.000000 	Validation Accuracy: 64.296296
Epoch 3/10000.. 
Training Loss: 0.742462 	Validation Loss: 0.751043 	Training Accuracy: 0.000000 	Validation Accuracy: 65.185185
EarlyStopping counter: 1 out of 20
Epoch 4/10000.. 
Training Loss: 0.726065 	Validation Loss: 0.754216 	Training Accuracy: 0.000000 	Validation Accuracy: 65.037037
EarlyStopping counter: 2 out of 20
Epoch 5/10000.. 
Training Loss: 0.713294 	Validation Loss: 0.757160 	Training Accuracy: 0.000000 	Validation Accuracy: 65.185185
EarlyStopping counter: 3 out of 20
Epoch 6/10000.. 
Training Loss: 0.702615 	Validation Loss: 0.759658 	Training Accuracy: 0.000000 	Validation Accuracy: 65.185185
EarlyStopping counter: 4 out of 20
Epoch 7/10000.. 
Training Loss: 0.693306 	Validation Loss: 0.761758 	Training Accuracy

In [17]:
def test(args, loader, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.eval()
    pred = []
    true = []
    soft = []
    for step, (x, y) in enumerate(loader):
        model.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        outputs = model(x)
        loss = criterion(outputs, y)
        
        # for majority voting
        softmax = torch.nn.Softmax(dim=1)
        s = softmax(outputs).cpu().detach().tolist()
        for i in range(len(s)):
            soft.append(s[i])

        predicted = outputs.argmax(1)
        preds = predicted.cpu().numpy()
        labels = y.cpu().numpy()
        preds = np.reshape(preds, (len(preds), 1))
        labels = np.reshape(labels, (len(preds), 1))

        for i in range(len(preds)):
            pred.append(preds[i][0].item())
            true.append(labels[i][0].item())
        
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss_epoch += loss.item()

    cnf_matrix = confusion_matrix(true, pred)
    print('Confusion Matrix:\n', cnf_matrix)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    accuracy_epoch = np.diag(cnf_matrix).sum().item() / len(true)
    
    # Specificity or true negative rate
    specificity = TN/(TN+FP) 

    print_specificity(specificity)

    report = classification_report(true, pred, target_names=['covid', 'healthy', 'others'])
    print(report)

    return loss_epoch, accuracy_epoch, (pred, true, soft)

def print_specificity(specificity):
    print('\t\tspecificity')
    print('')

    print(f'       covid\t{specificity[0]:.2f}')
    print(f'     healthy\t{specificity[1]:.2f}')
    print(f'      others\t{specificity[2]:.2f}')
    print('')

    macro_specificity = sum(specificity) / 3.0
    print(f'   macro avg\t{macro_specificity:.2f}')

    weighted = [434/835, 152/835, 249/835] 
    weighted_specificity = weighted @ specificity
    print(f'weighted avg\t{weighted_specificity:.2f}')
    print('')

In [18]:
# final testing
loss_epoch, accuracy_epoch, result = test(
    args, arr_test_loader, model, criterion, optimizer
)
print(
    f"[FINAL]\t Loss: {loss_epoch / len(arr_test_loader)}\t Accuracy: {accuracy_epoch}"
)

Confusion Matrix:
 [[342   9  51]
 [ 24  92  17]
 [155  30  56]]
		specificity

       covid	0.52
     healthy	0.94
      others	0.87

   macro avg	0.78
weighted avg	0.70

              precision    recall  f1-score   support

       covid       0.66      0.85      0.74       402
     healthy       0.70      0.69      0.70       133
      others       0.45      0.23      0.31       241

    accuracy                           0.63       776
   macro avg       0.60      0.59      0.58       776
weighted avg       0.60      0.63      0.60       776

[FINAL]	 Loss: 0.7204628729820252	 Accuracy: 0.6314432989690721


In [19]:
import os
import csv

preds, true, soft = result
images_path = test_loader.dataset.samples
# images_path -> [ [images path, label] * 835 ]

with open(f"majority{args.model_num}.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerow(["file", "prob_0", "prob_1", "prob_2", "pred", "label"])
    for i in range(len(preds)):
        f = os.path.basename(images_path[i][0])
        prob_0 = round(soft[i][0], 6)
        prob_1 = round(soft[i][1], 6)
        prob_2 = round(soft[i][2], 6)
        pred = preds[i]
        label = true[i]
        wr.writerow([f, prob_0, prob_1, prob_2, pred, label])    